In [1]:
from contextlib import contextmanager
import signal
import time
from tqdm.auto import tqdm
import warnings

from htools import hdir, count_calls, TimeExceededError

In [2]:
@contextmanager
def timebox(seconds):
#     @count_calls
    def update_custom_pbar(time_, frame):
        pbar.update(n=1)
        signal.alarm(1)
#         if update_custom_pbar.calls == time:
        if time.time() - pbar.start_t >= seconds:
            raise TimeExceededError('Time limit exceeded.')
    
    pbar = tqdm(total=seconds)
    try:
        signal.signal(signal.SIGALRM, update_custom_pbar)
        signal.alarm(1)
        yield
    except TimeExceededError as e:
        raise
    finally:
        pbar.close()
        signal.alarm(0)

In [3]:
with timebox(3.5) as tb:
    for i in range(20):
        time.sleep(0.2)
        j = i+1
        if i % 2 == 0: print(i)
    print(j)

0
2
4
6
8
10
12
14
16
18



TimeExceededError: Time limit exceeded.

In [18]:
@contextmanager
def i_timebox(seconds, strict=True, freq=.1):
    def update_custom_pbar(signum, frame):
        pbar.update(n=freq)
        if time.time() - pbar.start_t >= seconds:
            raise TimeExceededError('Time limit exceeded.')
    
    pbar = tqdm(total=seconds, bar_format='{l_bar}{bar}|{n:.2f}/{total:.1f}s')
    try:
        signal.signal(signal.SIGALRM, update_custom_pbar)
        signal.setitimer(signal.ITIMER_REAL, freq, freq)
        yield
    except TimeExceededError as e:
        if strict: raise
        warnings.warn(e.args[0])
    finally:
        pbar.close()
        signal.alarm(0)

In [19]:
with i_timebox(3):
    for i in range(35):
        n = [i+j for j in range(5_000_000)]
        print(n[:5])
        n = None

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]



TimeExceededError: Time limit exceeded.

In [20]:
with i_timebox(3, False):
    for i in range(35):
        n = [i+j for j in range(5_000_000)]
        print(n[:5])
        n = None
print('after')

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]

after


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Time limit exceeded.
  from ipykernel import kernelapp as app


In [9]:
def itimer_handler(signum, frame):
    print(signum, type(frame))

In [10]:
signal.signal(signal.SIGALRM, itimer_handler)
signal.setitimer(signal.ITIMER_REAL, 1, .5)

(0.0, 0.0)

In [10]:
signal.alarm(0)

0

In [1]:
from htools import timebox, timeboxed

In [18]:
@timebox(3.5)
def foo(a):
    for i in range(5_000_000):
        res = [a+j for j in range(100)]
        if i % 100_000 == 0: print(res[:3])
    return res

In [19]:
res = foo(34)

[34, 35, 36]
[34, 35, 36]
[34, 35, 36]



TimeExceededError: Time limit exceeded.